<a href="https://colab.research.google.com/github/Speedbird45Bravo/rando_projects/blob/main/NLTK_51221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [259]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from string import punctuation
import gensim
import gensim.downloader as api
import nltk
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential
stopwords = nltk.corpus.stopwords.words('english')

In [260]:
text = pd.read_csv("spam.csv", encoding="latin-1")
text = text[['v1','v2']]
text.columns = ['label','text']

In [261]:
def cleaner(text):
    text = "".join([char for char in text if char not in punctuation])
    tokens = re.split("\W+", text.lower())
    text = [word for word in tokens if word not in stopwords]
    return text

In [262]:
tfidf = TfidfVectorizer(analyzer=cleaner)

In [263]:
X = tfidf.fit_transform(text['text'])

In [264]:
X = pd.DataFrame(X.toarray())
y = text[['label']].copy()

In [265]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [266]:
rf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [267]:
predictions = rf.predict(X_test)

In [268]:
pos = "ham"
precision = precision_score(y_test, predictions, pos_label=pos).round(3)
recall = recall_score(y_test, predictions, pos_label=pos).round(3)

In [269]:
print("Precision: {} | Recall: {}".format(precision, recall))

Precision: 0.973 | Recall: 1.0


In [270]:
wiki_embeddings = api.load('glove-wiki-gigaword-100')

In [271]:
wiki_embeddings.most_similar('maine')

[('vermont', 0.8391201496124268),
 ('connecticut', 0.7976082563400269),
 ('oregon', 0.7730393409729004),
 ('massachusetts', 0.7728150486946106),
 ('missouri', 0.7665733098983765),
 ('virginia', 0.7642097473144531),
 ('rhode', 0.7578915357589722),
 ('carolina', 0.756413996219635),
 ('delaware', 0.7552822828292847),
 ('wisconsin', 0.7548272609710693)]

In [272]:
text['text_clean'] = text['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
X_train, X_test, y_train, y_test = train_test_split(text['text_clean'], text['label'])

In [273]:
w2v = gensim.models.Word2Vec(X_train, size=100, window=5, min_count=2)

In [274]:
w2v.wv.most_similar('gas')

[('oso', 0.9986931681632996),
 ('com', 0.9986872673034668),
 ('long', 0.9986636638641357),
 ('him', 0.9986581206321716),
 ('lol', 0.9986538887023926),
 ('off', 0.9986532926559448),
 ('cos', 0.9986474514007568),
 ('pobox', 0.9986388683319092),
 ('school', 0.9986297488212585),
 ('word', 0.9986221790313721)]

In [276]:
w2v_vect = np.array([np.array([w2v.wv[i] for i in ls if i in w2v.wv.index2word]) for ls in X_test])

11 10
9 9
7 7
11 11
7 6
6 6
4 3
8 8
9 9
27 26
15 15
13 11
19 17
5 4
4 4
8 8
21 19
6 6
23 23
6 6
18 16
5 5
13 13
18 16
24 20
6 6
25 21
18 16
11 11
13 11
6 6
15 12
9 8
9 8
6 6
15 12
22 18
32 31
11 11
3 3
12 12
7 7
22 21
19 16
5 5
16 12
23 17
8 7
7 5
25 20
7 6
27 27
8 8
13 11
24 24
13 13
22 20
5 4
27 27
3 3
8 8
7 6
4 4
21 20
16 16
23 19
22 20
3 3
18 15
6 5
6 6
20 19
5 5
23 21
7 6
21 21
21 17
13 13
25 23
3 3
25 25
23 23
31 28
15 13
17 17
6 6
4 4
26 23
30 28
9 9
31 28
14 14
7 7
30 23
25 25
6 6
10 10
12 12
25 16
31 29
8 7
20 19
8 7
7 7
50 44
15 14
8 8
1 1
7 7
9 7
10 9
7 7
13 12
23 22
21 19
26 26
7 7
13 12
28 27
1 1
4 3
44 41
8 8
4 4
7 6
15 13
10 9
13 12
5 4
4 4
10 10
14 12
12 10
24 19
11 10
18 17
6 6
19 18
27 6
5 4
9 9
29 29
19 14
20 20
8 8
9 9
24 24
6 6
8 7
4 3
8 8
12 12
19 16
5 5
14 13
9 9
10 10
4 4
23 21
11 10
8 8
24 17
5 5
6 5
42 42
5 5
10 10
8 8
31 29
19 17
7 6
7 5
6 6
21 13
28 25
6 6
12 12
22 21
5 5
1 0
13 13
30 29
10 8
12 12
26 19
10 10
25 25
26 22
8 6
14 11
23 22
5 3
12 12
8 7
9 8
3 

In [277]:
w2v_vect_avg = []
for vect in w2v_vect: 
    if len(vect)!=0:
        w2v_vect_avg.append(vect.mean(axis=0))
    else:
        w2v_vect_avg.append(np.zeros(100))

In [278]:
for i, v in enumerate(w2v_vect_avg):
    print(len(X_test.iloc[i]), len(v))

11 100
9 100
7 100
11 100
7 100
6 100
4 100
8 100
9 100
27 100
15 100
13 100
19 100
5 100
4 100
8 100
21 100
6 100
23 100
6 100
18 100
5 100
13 100
18 100
24 100
6 100
25 100
18 100
11 100
13 100
6 100
15 100
9 100
9 100
6 100
15 100
22 100
32 100
11 100
3 100
12 100
7 100
22 100
19 100
5 100
16 100
23 100
8 100
7 100
25 100
7 100
27 100
8 100
13 100
24 100
13 100
22 100
5 100
27 100
3 100
8 100
7 100
4 100
21 100
16 100
23 100
22 100
3 100
18 100
6 100
6 100
20 100
5 100
23 100
7 100
21 100
21 100
13 100
25 100
3 100
25 100
23 100
31 100
15 100
17 100
6 100
4 100
26 100
30 100
9 100
31 100
14 100
7 100
30 100
25 100
6 100
10 100
12 100
25 100
31 100
8 100
20 100
8 100
7 100
50 100
15 100
8 100
1 100
7 100
9 100
10 100
7 100
13 100
23 100
21 100
26 100
7 100
13 100
28 100
1 100
4 100
44 100
8 100
4 100
7 100
15 100
10 100
13 100
5 100
4 100
10 100
14 100
12 100
24 100
11 100
18 100
6 100
19 100
27 100
5 100
9 100
29 100
19 100
20 100
8 100
9 100
24 100
6 100
8 100
4 100
8 100
12 100
19

In [279]:
tagged_docs = [gensim.models.doc2vec.TaggedDocument(v, [i]) for i, v in enumerate(X_train)]

In [280]:
d2v = gensim.models.Doc2Vec(tagged_docs, vector_size=100, window=5, min_count=2)

In [281]:
d2v.infer_vector(['learning','nlp'])

array([ 1.1585198e-03, -1.7702891e-03, -3.9783851e-03, -3.3107155e-03,
       -3.2312365e-03,  2.2073989e-03,  3.2686206e-04,  6.4054033e-04,
        4.7178618e-03, -4.5176796e-03, -1.8538650e-03,  3.2499733e-03,
        2.1692119e-03,  5.5705057e-04,  7.4737793e-04, -1.6829991e-03,
        9.1254566e-04, -4.7547063e-03, -3.8845546e-03, -5.3367543e-04,
        1.4299899e-03,  1.5727771e-03,  1.9610636e-03,  2.3519578e-03,
        4.2742407e-03, -4.4737474e-04, -1.2050465e-03,  3.1389545e-03,
        1.7138439e-03, -9.4149861e-04,  1.3699711e-03,  1.2977759e-03,
        2.2326328e-03, -3.6665742e-04,  4.6535963e-03, -6.6216389e-04,
        3.7814227e-03, -3.0363260e-03,  1.2784472e-03, -3.1920576e-03,
        1.3483056e-03, -4.2847558e-03,  8.2621380e-04, -9.0612361e-04,
        4.3056454e-03, -4.8066471e-03, -9.3716563e-04, -4.9531730e-03,
        4.4374391e-03,  2.0413743e-03,  4.3739605e-04, -1.2739189e-03,
       -1.8782100e-03, -2.5141225e-03,  2.1704307e-03, -3.5871760e-04,
      

In [282]:
vectors = [[d2v.infer_vector(words)] for words in X_test]

In [283]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [284]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [285]:
X_train_sequences_padded = pad_sequences(X_train_sequences, 50)
X_test_sequences_padded = pad_sequences(X_test_sequences, 50)

In [291]:
y_train = y_train.replace("ham",1)
y_train = y_train.replace("spam",0)

In [292]:
y_test = y_test.replace("ham",1)
y_test = y_test.replace("spam",0)

In [293]:
y_train.value_counts()

1    3617
0     562
Name: label, dtype: int64

In [294]:
y_test

1320    1
3251    1
815     1
182     1
5042    1
       ..
3636    1
4098    1
445     1
4268    1
2241    1
Name: label, Length: 1393, dtype: int64

In [295]:
def recall_score(y_true, y_predictions):
  true_positives = K.sum(K.round(K.clip(y_true * y_predictions, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

In [296]:

def precision_score(y_true, y_predictions):
  true_positives = K.sum(K.round(K.clip(y_true * y_predictions, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_predictions, 0, 1)))
  recall = true_positives / (predicted_positives + K.epsilon())
  return recall

In [297]:
model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(32, dropout=0, recurrent_dropout=0))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 32)          214496    
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 223,905
Trainable params: 223,905
Non-trainable params: 0
_________________________________________________________________


In [298]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [300]:
history = model.fit(X_train_sequences_padded, y_train, batch_size=32, epochs=10, validation_data=(X_test_sequences_padded, y_test))

Epoch 1/10
131/131 [==============================] - 7s 33ms/step - loss: 0.4884 - accuracy: 0.8607 - val_loss: 0.1117 - val_accuracy: 0.9742
Epoch 2/10
131/131 [==============================] - 4s 29ms/step - loss: 0.0572 - accuracy: 0.9856 - val_loss: 0.0517 - val_accuracy: 0.9864
Epoch 3/10
131/131 [==============================] - 4s 28ms/step - loss: 0.0159 - accuracy: 0.9953 - val_loss: 0.0580 - val_accuracy: 0.9878
Epoch 4/10
131/131 [==============================] - 4s 28ms/step - loss: 0.0035 - accuracy: 0.9992 - val_loss: 0.0705 - val_accuracy: 0.9871
Epoch 5/10
131/131 [==============================] - 4s 28ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.0694 - val_accuracy: 0.9871
Epoch 6/10
131/131 [==============================] - 4s 29ms/step - loss: 5.0605e-04 - accuracy: 1.0000 - val_loss: 0.0759 - val_accuracy: 0.9864
Epoch 7/10
131/131 [==============================] - 4s 28ms/step - loss: 1.9994e-04 - accuracy: 1.0000 - val_loss: 0.0841 - val_accuracy

In [305]:
predictions = model.predict(X_test_sequences_padded).round(0)

In [310]:
loss, accuracy = model.evaluate(y_test, predictions)
print("Test Accuracy: {}%".format(round(accuracy * 100),))

44/44 [==============================] - 0s 2ms/step - loss: 0.4619 - accuracy: 0.8794
Test Accuracy: 88%
